In [8]:
import PyPDF2
from tqdm import tqdm
import spacy
from collections import defaultdict
import pycountry
import os
import numpy as np
import math
import json

In [9]:
pdfs = [file for file in os.listdir("pdf") if not file.startswith(".") ]
documents = {} 

for pdf in pdfs:
    pdf_path = "pdf/" + pdf
    txt_file = pdf[:-3] + "txt"
    txt_path = "txt/" + txt_file
    
    if txt_file in os.listdir("txt"):
        print("File already transformed to txt")
        with open(txt_path, 'r', encoding='utf-8') as f:
            text = f.read()
    else: 
        text = ""
        with open(pdf_path, 'rb') as file:
            reader = PyPDF2.PdfReader(file)
            for page in tqdm(reader.pages):
                text += page.extract_text() + "PAGE END"
                
        text = text.replace('\n', '')
        with open(txt_path, 'w', encoding='utf-8') as f:
            f.write(text)

    documents[pdf] = text

File already transformed to txt
File already transformed to txt
File already transformed to txt
File already transformed to txt
File already transformed to txt
File already transformed to txt
File already transformed to txt
File already transformed to txt
File already transformed to txt
File already transformed to txt
File already transformed to txt
File already transformed to txt
File already transformed to txt
File already transformed to txt
File already transformed to txt
File already transformed to txt


100%|█████████████████████████████████████████████████████████████████████████████████████████████| 113/113 [00:04<00:00, 24.90it/s]


File already transformed to txt
File already transformed to txt
File already transformed to txt
File already transformed to txt
File already transformed to txt
File already transformed to txt
File already transformed to txt
File already transformed to txt


In [10]:
documents.keys()

dict_keys(['Stellantis.pdf', 'Shell.pdf', 'AIG.pdf', 'Siemens.pdf', 'SAP.pdf', 'Maersk.pdf', 'Volkswagen.pdf', 'Nike.pdf', 'Aramco.pdf', 'NovoNordisk.pdf', 'CocaCola.pdf', 'Toyota.pdf', 'Tesla.pdf', 'Boeing.pdf', 'Samsung.pdf', 'Nestle.pdf', 'Apple.pdf', 'Eni.pdf', 'BASF.pdf', 'P&G.pdf', 'Nvidia (sustainability).pdf', 'Alibaba.pdf', 'TotalEnergies.pdf', 'ExxonMobil_P1.pdf', 'LVMH.pdf'])

In [13]:
total_tokens = 0
for i, doc in enumerate(documents.values()):
    tokens = doc.split()
    print(f"Document {i}: {len(tokens)} tokens")
    total_tokens += len(tokens)

Document 0: 222011 tokens
Document 1: 312960 tokens
Document 2: 130543 tokens
Document 3: 93710 tokens
Document 4: 164942 tokens
Document 5: 110234 tokens
Document 6: 319521 tokens
Document 7: 56932 tokens
Document 8: 105256 tokens
Document 9: 91239 tokens
Document 10: 94012 tokens
Document 11: 97541 tokens
Document 12: 59441 tokens
Document 13: 42628 tokens
Document 14: 138173 tokens
Document 15: 16249 tokens
Document 16: 50729 tokens
Document 17: 256985 tokens
Document 18: 89544 tokens
Document 19: 52813 tokens
Document 20: 14549 tokens
Document 21: 177211 tokens
Document 22: 407312 tokens
Document 23: 2516 tokens
Document 24: 28986 tokens


In [14]:
total_tokens

3136037

In [7]:
metroman = []

for key in list(documents.keys()):
    # Define document
    print("Define document")
    current_doc = documents[key]
    doc_len = len(current_doc.split())

    # Indexes
    print("Define indexes")
    if doc_len < 100000:
        indexes = None
    else:
        n_steps = math.ceil(doc_len / 100000)
        indexes = list(np.linspace(0, doc_len, n_steps))
        indexes = [round(item) for item in indexes]

    # NER
    print("Current Indexes:", indexes)
    nlp = spacy.load("en_core_web_sm") 
    
    valid_countries = {country.name for country in pycountry.countries}
    country_count = defaultdict(int)

    if indexes == None:
        print("small document")
        ner_model = nlp(current_doc)
        for ent in tqdm(ner_model.ents):
            if ent.label_ == "GPE" and ent.text.strip() in valid_countries:
                country = ent.text.strip()
                country_count[country] += 1
    else:
        print("big document")
        for j in range(len(indexes)-1):
            start = indexes[j]
            end = indexes[j+1]
            ner_doc = current_doc[start:end]
            ner_model = nlp(ner_doc)
            for ent in tqdm(ner_model.ents):
                if ent.label_ == "GPE" and ent.text.strip() in valid_countries:
                    country = ent.text.strip()
                    country_count[country] += 1

    metroman.append(country_count)
    with open(f'json/{key}.txt', 'w') as f:
        json.dump(country_count, f)

Define document
Define indexes
Current Indexes: [0, 111006, 222011]


/Users/fabiostefana/anaconda3/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


big document


100%|███████████████████████████████████████████████████████████████████████████████████████| 1712/1712 [00:00<00:00, 762600.73it/s]


Define document
Define indexes
Current Indexes: [0, 104320, 208640, 312960]
big document


...

100%|██████████████████████████████████████████████████████████████████████████████████████| 2744/2744 [00:00<00:00, 3232912.97it/s]


Define document
Define indexes
Current Indexes: None
small document


100%|█████████████████████████████████████████████████████████████████████████████████████████| 151/151 [00:00<00:00, 137652.66it/s]


Define document
Define indexes
Current Indexes: None
small document


100%|██████████████████████████████████████████████████████████████████████████████████████| 3450/3450 [00:00<00:00, 3734283.56it/s]


In [ ]:
metrocity = []
for dick in metroman:
    x = dict(sorted(dick.items(), key=lambda item: item[1], reverse=True)[:10])
    metrocity.append(x)

In [ ]:
metrocity